In [261]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import gensim
from gensim import corpora, models, similarities
import re
from gensim.models.ldamodel import LdaModel
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
import matplotlib.colors as mcolors
import seaborn as sns
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.test.utils import datapath

In [2]:
stop_words = stopwords.words('english')

In [3]:
df = pd.read_excel("./tweets.xlsx")

In [4]:
df['tweets'] = df['text']

In [5]:
def tokenize(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub('@\S*\s?', '', text)  # remove tags
    text = re.sub('\s+', ' ', text)  # remove newline chars
    text = re.sub("\'", "", text)  # remove single quotes
    text = gensim.utils.simple_preprocess(str(text), deacc=True) 
    #tokens = word_tokenize(text)
    stopwords_removed = [token.lower() for token in text if token.lower() not in stop_words and len(token) > 3]
    return stopwords_removed

df['text'] = df['text'].apply(tokenize)


<>:3: DeprecationWarning: invalid escape sequence '\S'
<>:4: DeprecationWarning: invalid escape sequence '\s'
<>:3: DeprecationWarning: invalid escape sequence '\S'
<>:4: DeprecationWarning: invalid escape sequence '\s'
C:\Users\Arushi Jain\AppData\Local\Temp\ipykernel_41268\1660415116.py:3: DeprecationWarning: invalid escape sequence '\S'
  text = re.sub('@\S*\s?', '', text)  # remove tags
C:\Users\Arushi Jain\AppData\Local\Temp\ipykernel_41268\1660415116.py:4: DeprecationWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text)  # remove newline chars


In [216]:
lemmatizer = WordNetLemmatizer()
def lemmatize_text(df_text):
    lemmatized =[]
    for w in df_text:
        lemmatized.append(lemmatizer.lemmatize(w))
    more_stopwords = ['wasnt','cant','send', 'didnt','much','youve','able', 'still','maybe', 'dont', 'im', 'wont', 'youre', 'le', 'say', 'sure', 'way', 'help', 'need', 'america', 'many', 'back', 'u.s.', '&amp' ]

    stopwords_removed = [token for token in lemmatized if token not in more_stopwords and len(token) > 3]

    return stopwords_removed

df['text'] = df['text'].apply(lemmatize_text)


In [217]:
df['text']

0                             [smalbiz, backbone, economy, right, democrat, getting, crushed, bidenflation]
1       [today, opportunity, attend, ladiesforlaxalt, business, roundtable, small, business, backbone, s...
2       [great, meet, team, mcon, civil, construction, vega, juan, started, mcon, grown, company, employ...
3       [naral, called, defunding, police, month, later, accepted, endorsement, masto, went, naral, inva...
4       [great, time, talking, meeting, supporter, lola, restaurant, vega, also, ethan, high, school, se...
                                                       ...                                                 
9616                                                            [check, lashondra, tinsleys, video, tiktok]
9617    [vote, november, votevotevote, vote, voteforme, voteforchange, candidate, writein, ussenate, bet...
9618    [join, several, candidate, write, endorsed, saturday, october, cleveland, ohio, panel, discussio...
9619    [forget, tell, frien

In [234]:
# Create Unigrams and Bigrams
texts = list(df['text'])
dictionary = corpora.Dictionary(texts)
# bigram = Phrases(texts, min_count=1, threshold=2)
# bigram_phraser = Phraser(bigram)
# bigram_token = []
# for sent in texts:
#     bigram_token.append(bigram_phraser[sent])
# dictionary = corpora.Dictionary(bigram_token)

In [235]:
dictionary.save_as_text("dict_text.txt") 
data = pd.read_csv('dict_text.txt') 
splits_l = []
for index, d in data.iterrows():
    splits = d.str.split('\t')[0]
    splits_d = {"token_id":splits[0], "token_name":splits[1], "token_frequency":splits[2]}
    splits_l.append(splits_d)
token_df = pd.DataFrame(splits_l)


In [236]:
token_df.sort_values("token_frequency", ascending=False)

,token_id,token_name,token_frequency
11340,543,worker,99
7409,966,paying,99
7092,313,open,99
2186,710,control,98
4198,1142,funding,97
...,...,...,...
4179,5287,fucking,1
4183,10431,fuerzamexico,1
4184,3777,fugicles,1
4187,6143,fullest,1


In [237]:
dictionary.filter_extremes(no_below = 5)
dictionary.filter_extremes(no_above = 0.8)

In [238]:
dictionary.save_as_text("dict_text.txt") 
data = pd.read_csv('dict_text.txt') 
splits_l = []
for index, d in data.iterrows():
    splits = d.str.split('\t')[0]
    splits_d = {"token_id":splits[0], "token_name":splits[1], "token_frequency":splits[2]}
    splits_l.append(splits_d)
token_df = pd.DataFrame(splits_l)

In [239]:
token_df

,token_id,token_name,token_frequency
0,1666,abandoned,6
1,2315,abhorrent,5
2,2556,ability,24
3,720,abortion,182
4,1850,absent,5
...,...,...,...
3325,884,young,73
3326,935,youth,16
3327,1250,yuma,7
3328,1190,zero,15


In [249]:
corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus] 

In [250]:
total_topics = 20

In [255]:
lda = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=total_topics, random_state=1)
corpus_lda = lda[corpus_tfidf]

In [256]:
lda.show_topics(total_topics)

[(0,
  '0.015*"energy" + 0.011*"information" + 0.010*"strong" + 0.010*"recovery" + 0.009*"issue" + 0.008*"american" + 0.008*"earn" + 0.008*"kendra" + 0.007*"expected" + 0.007*"creating"'),
 (1,
  '0.015*"vote" + 0.014*"medium" + 0.012*"watch" + 0.011*"enjoyed" + 0.010*"herschel" + 0.010*"sorry" + 0.009*"word" + 0.009*"walker" + 0.009*"senate" + 0.009*"endorsement"'),
 (2,
  '0.010*"sign" + 0.009*"something" + 0.008*"inflation" + 0.008*"watching" + 0.008*"independent" + 0.008*"lower" + 0.008*"energy" + 0.007*"facebook" + 0.007*"saying" + 0.007*"letter"'),
 (3,
  '0.012*"social" + 0.011*"cost" + 0.011*"medicare" + 0.010*"security" + 0.010*"senior" + 0.009*"insurance" + 0.009*"discus" + 0.009*"prescription" + 0.008*"farmer" + 0.008*"insulin"'),
 (4,
  '0.008*"corporation" + 0.008*"easy" + 0.007*"profit" + 0.007*"based" + 0.007*"created" + 0.007*"resolution" + 0.007*"location" + 0.007*"parade" + 0.006*"dear" + 0.006*"side"'),
 (5,
  '0.015*"abortion" + 0.014*"woman" + 0.014*"choice" + 0.01

In [257]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    sent_topics_df["original"] = df['tweets']
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda, corpus=corpus_tfidf, texts=texts)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text', 'Original']

df_dominant_topic[['Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Original']].head(40)


C:\Users\Arushi Jain\AppData\Local\Temp\ipykernel_41268\963654183.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


,Dominant_Topic,Topic_Perc_Contrib,Keywords,Original
0,5,0.3910,"abortion, woman, choice, libertarian, freedom, health, care, right, access, puerto","RT @joniernst: #SmalBiz are the backbone of our economy, and right now under Democrats they’re g..."
1,10,0.4737,"together, debate, done, candidate, party, democracy, would, fantastic, weekend, politics",Today I had the opportunity to attend a #LadiesForLaxalt Business Roundtable with @joniernst &am...
2,14,0.6963,"fetterman, wait, incredible, interview, center, john, running, conversation, business, trust",It was great to meet the team at MCON Civil Construction in Las Vegas.\n\nJuan started MCON in 2...
3,2,0.2605,"sign, something, inflation, watching, independent, lower, energy, facebook, saying, letter","NARAL called for the defunding of police &amp; 4 months later, @CortezMasto accepted their endor..."
4,18,0.3786,"hurricaneian, florida, great, night, update, following, game, safe, mike, storm",I had a great time talking and meeting with supporters at Lola’s restaurant in Las Vegas. \n\nI ...
5,2,0.6515,"sign, something, inflation, watching, independent, lower, energy, facebook, saying, letter",RT @GOP: Nevadans need leader @AdamLaxalt to represent them in the Senate. \n \n@GOPChairwoman d...
6,0,0.6648,"energy, information, strong, recovery, issue, american, earn, kendra, expected, creating","RT @AdamLaxalt: With a mouth full of ice cream, Biden says our economy is “strong as hell.”\n\nW..."
7,0,0.6758,"energy, information, strong, recovery, issue, american, earn, kendra, expected, creating","With a mouth full of ice cream, Biden says our economy is “strong as hell.”\n\nWhile Nevadans su..."
8,14,0.5709,"fetterman, wait, incredible, interview, center, john, running, conversation, business, trust",Wrapped up another great meet and greet in Reno this afternoon. \n\nI’m continually amazed by th...
9,0,0.6559,"energy, information, strong, recovery, issue, american, earn, kendra, expected, creating",It was great to reflect on Nevada’s rich history with so many at the Silver Springs Founders Day...


In [259]:
df_dominant_topic.to_excel('./topics.xlsx', index=False)

In [266]:
lda.save('lda.model')

In [260]:
sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text", "Original"]

# Show
sent_topics_sorteddf_mallet

,Topic_Num,Topic_Perc_Contrib,Keywords,Representative Text,Original
0,0,0.8028,"energy, information, strong, recovery, issue, american, earn, kendra, expected, creating","[president, biden, looked, everywhere, else, energy, instead, home, price, working, american, un...",President Biden has looked everywhere else for energy instead of here at home. \n\nWho pays the ...
1,1,0.7876,"vote, medium, watch, enjoyed, herschel, sorry, word, walker, senate, endorsement","[said, debate, drug, cartel, member, could, vote, would, vote, mark, kelly, exaggerating, biden,...","When I said in our debate that if the drug cartel members could vote, they would all vote for Ma..."
2,2,0.8190,"sign, something, inflation, watching, independent, lower, energy, facebook, saying, letter","[working, american, already, paying, price, michael, bennet, bidens, reckless, spending, getting...",Working Americans have already been paying the price for Michael Bennet and Joe Biden's reckless...
3,3,0.7955,"social, cost, medicare, security, senior, insurance, discus, prescription, farmer, insulin","[absent, spent, time, think, avoid, debate, convince, voter, reelect, senate, town, hall, meetin...",Absent @SenatorHassan has spent too much time in DC if she thinks she can avoid debates &amp; st...
4,4,0.8009,"corporation, easy, profit, based, created, resolution, location, parade, dear, side","[ready, debate, issue, know, told, absolutely, past, saturday, people, oklahoma, deserve, better...",".@SenatorLankford I am here and ready to debate you on the issues. I know you told us ""absolutel..."
5,5,0.7898,"abortion, woman, choice, libertarian, freedom, health, care, right, access, puerto","[reproductive, health, care, decision, made, patient, politician, always, fight, support, woman,...","Reproductive health care decisions should be made by patients, not politicians. I will always fi..."
6,6,0.8059,"georgia, saturday, legislation, beautiful, seeing, reduction, retweet, housing, sense, looking","[state, senator, fight, marriage, equality, keep, standing, lgbtq, american, proud, earned, scor...","As a state senator, I led the fight for marriage equality in 2009, and I will keep standing with..."
7,7,0.7562,"ryan, county, live, candidate, join, saudi, debate, tonight, matter, heading","[sharon, well, wish, jewish, friend, granite, state, across, observing, kippur]",Sharon and I send well wishes to our Jewish friends in the Granite State and across America obse...
8,8,0.8291,"chase, police, border, terrorist, black, criminal, crime, statement, southern, enforcement","[name, cheri, beasley, former, public, defender, first, black, woman, serve, chief, justice, nor...","My name is Cheri Beasley. \n\nI’m a former public defender, and the first Black woman to serve a..."
9,9,0.8027,"register, check, vote, mike, loan, thanks, team, response, good, october","[north, carolina, left, register, vote, online, visit, make, plan, vote, election, also, registe...","🗳️North Carolina, you have 1️⃣ day left to register to vote online! \n\nVisit https://t.co/XqGbc..."


In [267]:
pyLDAvis.enable_notebook()

In [268]:
panel = pyLDAvis.gensim_models.prepare(lda, corpus_lda, dictionary)
panel


C:\Users\Arushi Jain\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.055713  0.003238       1        1  16.015923
14     0.008400  0.037416       2        1  13.214882
5     -0.025448 -0.059920       3        1   8.960616
2     -0.033119  0.083232       4        1   8.212031
3     -0.047526  0.057524       5        1   5.647991
15    -0.013763  0.027848       6        1   4.468039
8     -0.080684  0.034614       7        1   3.988492
17    -0.014088 -0.013631       8        1   3.832777
12    -0.081503  0.041032       9        1   3.636184
4     -0.057943  0.117454      10        1   3.480816
0     -0.065640  0.040083      11        1   3.223490
11    -0.034047 -0.055597      12        1   3.104579
6     -0.023484  0.039100      13        1   3.078701
1      0.029795 -0.071354      14        1   2.935570
9      0.071662  0.015050      15        1   2.723257
19    -0.061710 -0.077567      16        1   2.714459
18     0.093364  0.057229      17        1   2.710909
7      0.269314  0.048342      18        1   2.683761
16    -0.031247 -0.133286      19        1   2.683761
13     0.041955 -0.190808      20        1   2.683760, topic_info=              Term       Freq      Total Category  logprob  loglift
155          thank  37.000000  37.000000  Default  30.0000  30.0000
2137     fetterman  23.000000  23.000000  Default  29.0000  29.0000
1498       florida  16.000000  16.000000  Default  28.0000  28.0000
1311       georgia  10.000000  10.000000  Default  27.0000  27.0000
2394  hurricaneian   8.000000   8.000000  Default  26.0000  26.0000
...            ...        ...        ...      ...      ...      ...
677          world   1.818981  12.143734  Topic20  -4.9442   1.7194
244          woman   1.897640  28.737686  Topic20  -4.9018   0.9004
1436       protect   1.609959  14.901071  Topic20  -5.0662   1.3927
155          thank   1.580925  37.153693  Topic20  -5.0844   0.4609
23           today   1.551391  35.703474  Topic20  -5.1033   0.4819

[1145 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2556      8  0.893704   ability
720       1  0.058996  abortion
720       2  0.058996  abortion
720       3  0.707953  abortion
720       4  0.058996  abortion
...     ...       ...       ...
1792      1  0.980762      york
884       1  0.164128     young
884       3  0.164128     young
884      12  0.164128     young
884      19  0.492385     young

[2415 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 15, 6, 3, 4, 16, 9, 18, 13, 5, 1, 12, 7, 2, 10, 20, 19, 8, 17, 14])

Sources: 
- https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/
- https://www.kaggle.com/code/errearanhas/topic-modelling-lda-on-elon-tweets
- https://towardsdatascience.com/lda-topic-modeling-with-tweets-deff37c0e131